<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/Retrievals_approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-core langchain-community --quiet
!pip install sentence_transformers FlagEmbedding chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 802.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 2

In [2]:
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
import chromadb
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.storage._lc_store import create_kv_docstore
from langchain.storage.file_system import LocalFileStore

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'}, #gpu
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
questions = pd.read_fwf("https://raw.githubusercontent.com/winterForestStump/thesis/main/questions/questions_ver2.txt", names=['question'])
company = "COCA COLA CO"
questions['question_name'] = questions['question'].str.replace('company', company)

# Approach #1

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as NOT part of the question
* Number of retrieved parent chunks = 2
* Distance function - Cosine similarity

In [6]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports", metadata={"hnsw:space": "cosine"})
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb'
    )
vectorstore.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [7]:
NUM_PAR_CHUNKS = 2

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company}, 'k': NUM_PAR_CHUNKS})

In [8]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question'][i])
    results_list.append(pd.DataFrame({
        'question': questions['question'][i],
        'context_cosine': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [04:01<00:00,  6.90s/it]


In [9]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach1.json')

# Approach #2

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as part of the question
* Number of retrieved parent chunks = 2
* Distance function - Cosine similarity

In [10]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question_name'][i])
    results_list.append(pd.DataFrame({
        'question_name': questions['question_name'][i],
        'context_cosine_name': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:40<00:00,  2.88s/it]


In [11]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach2.json')

# Approach #3

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as NOT part of the question
* Number of retrieved parent chunks = 2
* Distance function - Inner product

In [12]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports", metadata={"hnsw:space": "ip"})
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb'
    )
vectorstore.persist()

In [13]:
NUM_PAR_CHUNKS = 2

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company}, 'k': NUM_PAR_CHUNKS})

In [14]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question'][i])
    results_list.append(pd.DataFrame({
        'question': questions['question'][i],
        'context_ip': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:45<00:00,  3.02s/it]


In [15]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach3.json')

# Approach #4

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as part of the question
* Number of retrieved parent chunks = 2
* Distance function - Inner product

In [16]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question_name'][i])
    results_list.append(pd.DataFrame({
        'question_name': questions['question_name'][i],
        'context_ip_name': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:27<00:00,  2.51s/it]


In [17]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach4.json')

# Approach #5

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as NOT part of the question
* Number of retrieved parent chunks = 2
* Distance function - Squared L2

In [18]:
persistent_client = chromadb.PersistentClient('/content/drive/MyDrive/Thesis/chromadb')
collection = persistent_client.get_or_create_collection("Reports", metadata={"hnsw:space": "ip"})
fs = LocalFileStore('/content/drive/MyDrive/Thesis/reports_store_location')
store = create_kv_docstore(fs)
vectorstore = Chroma(client = persistent_client,
                     collection_name="Reports",
                     embedding_function=bge_embeddings,
                     persist_directory='/content/drive/MyDrive/Thesis/chromadb'
    )
vectorstore.persist()

In [19]:
NUM_PAR_CHUNKS = 2

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    search_kwargs={'filter': {'company': company}, 'k': NUM_PAR_CHUNKS})

In [20]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question'][i])
    results_list.append(pd.DataFrame({
        'question': questions['question'][i],
        'context_l2': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:27<00:00,  2.50s/it]


In [21]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach5.json')

# Approach #6

* ParentDocumnetRetriever
* Name of the company as a filter
* Name of the company as part of the question
* Number of retrieved parent chunks = 2
* Distance function - Squared L2

In [22]:
from tqdm import tqdm
results_list = []

for i in tqdm(range(len(questions))):
    response = big_chunks_retriever.invoke(questions['question_name'][i])
    results_list.append(pd.DataFrame({
        'question_name': questions['question_name'][i],
        'context_ip_name': response
    }))

results = pd.concat(results_list, ignore_index=True)

100%|██████████| 35/35 [01:27<00:00,  2.50s/it]


In [23]:
results.to_json('/content/drive/MyDrive/Thesis/retrievers/results_approach6.json')